In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [6]:
city = 'chicago'

In [7]:
#read in labels
labels = pd.read_csv('../../../data/' + city + '/processed-labels/labels_raw_' + city + '.csv')
labels

,audit_task_id,label_id,gsv_panorama_id_x,label_type,severity,correct,high_quality_user,geometry,gsv_panorama_id_y,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng
0,3,8,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,NaN,True,POINT (-87.9803466796875 42.08366775512695),pMph48Z6Xz2hexFJ7q3d_Q,2,136.687500,-18.625000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083668,-87.980347
1,3,9,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,1.0,True,True,POINT (-87.98060607910156 42.08369445800781),pMph48Z6Xz2hexFJ7q3d_Q,2,249.937500,-14.125000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083694,-87.980606
2,3,10,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,False,True,POINT (-87.98060607910156 42.08378601074219),pMph48Z6Xz2hexFJ7q3d_Q,3,288.656250,-13.187500,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083786,-87.980606
3,3,11,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,True,True,POINT (-87.98035430908203 42.08379364013672),pMph48Z6Xz2hexFJ7q3d_Q,3,49.245537,-16.220982,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083794,-87.980354
4,3,12,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,True,True,POINT (-87.98036193847656 42.08381652832031),pMph48Z6Xz2hexFJ7q3d_Q,3,42.636162,-16.033482,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083817,-87.980362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,4588,21507,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,3.0,True,True,POINT (-87.66429138183594 41.61117935180664),hhAF_qDTHgN0dL8SNsr-oA,1,126.341515,-32.937500,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611179,-87.664291
18281,4588,21508,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,2.0,NaN,True,POINT (-87.66455078125 41.61113739013672),hhAF_qDTHgN0dL8SNsr-oA,1,213.154022,-35.000000,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611137,-87.664551
18282,4589,21509,tZWhmOw4CIP9bQLJycSyqQ,CurbRamp,1.0,NaN,True,POINT (-87.6463394165039 41.60987091064453),tZWhmOw4CIP9bQLJycSyqQ,3,217.380630,-16.302500,179.386520,-0.660645,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,41.609871,-87.646339
18283,4601,21517,nWTcoxo4S0MImzzvlQGwpQ,CurbRamp,1.0,True,True,POINT (-87.65129089355469 41.62562561035156),nWTcoxo4S0MImzzvlQGwpQ,3,248.244995,-13.510625,359.491150,-0.079690,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,41.625626,-87.651291


In [8]:
#drop columns audit_task_id, correct, high_quality_user, gsv_panorama_id_y
labels = labels.drop(columns=['audit_task_id', 'correct', 'high_quality_user', 'gsv_panorama_id_y'])
#renname gsv_panorama_id_y to gsv_panorama_id
labels = labels.rename(columns={'gsv_panorama_id_x': 'gsv_panorama_id'})

In [9]:
#read in all files in the feature folder
clustered = pd.read_csv('../../../data/' + city + '/features/clustered.csv')
distance_to_road = pd.read_csv('../../../data/' + city + '/features/distance_to_road.csv')
distance_to_intersection = pd.read_csv('../../../data/' + city + '/features/distance_to_intersection.csv')
optional_input = pd.read_csv('../../../data/' + city + '/features/optional_input.csv')

In [10]:
#merge all into one on label_id
df = labels.merge(clustered, on='label_id').merge(distance_to_road, on='label_id').merge(distance_to_intersection, on='label_id').merge(optional_input, on='label_id')

In [11]:
#keep only the label types Curb Ramp, Missing Curb Ramp, Obstacle in Path, Surface Problem, and No Sidewalk
df = df[df['label_type'].isin(['CurbRamp', 'NoCurbRamp', 'Obstacle', 'SurfaceProblem', 'NoSidewalk'])]

In [12]:
#drop nan in severity
df = df.dropna(subset=['severity'])

In [13]:
df

,label_id,gsv_panorama_id,label_type,severity,geometry,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
0,8,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,POINT (-87.9803466796875 42.08366775512695),2,136.687500,-18.625000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083668,-87.980347,6904,0,1,residential,26.799369,43.883001,1.0,1.0,0.0
1,9,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,1.0,POINT (-87.98060607910156 42.08369445800781),2,249.937500,-14.125000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083694,-87.980606,3813,0,1,secondary,35.678794,40.207796,0.0,0.0,0.0
2,10,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,POINT (-87.98060607910156 42.08378601074219),3,288.656250,-13.187500,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083786,-87.980606,7014,0,1,secondary,36.434254,39.393508,1.0,1.0,0.0
3,11,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,POINT (-87.98035430908203 42.08379364013672),3,49.245537,-16.220982,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083794,-87.980354,3708,0,1,residential,19.033046,37.148259,1.0,2.0,0.0
4,12,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,POINT (-87.98036193847656 42.08381652832031),3,42.636162,-16.033482,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083817,-87.980362,7719,0,1,residential,27.337529,40.358872,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,21507,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,3.0,POINT (-87.66429138183594 41.61117935180664),1,126.341515,-32.937500,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611179,-87.664291,14528,0,1,residential,34.337291,0.000000,1.0,1.0,0.0
18281,21508,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,2.0,POINT (-87.66455078125 41.61113739013672),1,213.154022,-35.000000,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611137,-87.664551,14505,0,1,residential,36.532837,0.000000,1.0,1.0,0.0
18282,21509,tZWhmOw4CIP9bQLJycSyqQ,CurbRamp,1.0,POINT (-87.6463394165039 41.60987091064453),3,217.380630,-16.302500,179.386520,-0.660645,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,41.609871,-87.646339,516,1,3,residential,24.820652,45.526460,0.0,0.0,0.0
18283,21517,nWTcoxo4S0MImzzvlQGwpQ,CurbRamp,1.0,POINT (-87.65129089355469 41.62562561035156),3,248.244995,-13.510625,359.491150,-0.079690,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,41.625626,-87.651291,500,1,2,residential,13.633726,39.688388,0.0,0.0,0.0


In [14]:
gt = pd.read_csv('../../../data/' + city + '/features/' + city + '_ground_truth_labels.csv')

In [15]:
gt

,label_id,ground_truth
0,8,1
1,9,1
2,11,1
3,14,1
4,18,1
...,...,...
6251,21502,1
6252,21505,1
6253,21506,1
6254,21507,1


In [16]:
#merge gt with labels how left
test= pd.merge(df, gt, on='label_id', how='inner')

In [17]:
#rename ground truth to verified
test.rename(columns={'ground_truth': 'verified'}, inplace=True)

In [18]:
test

,label_id,gsv_panorama_id,label_type,severity,geometry,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description,verified
0,8,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,POINT (-87.9803466796875 42.08366775512695),2,136.687500,-18.625000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083668,-87.980347,6904,0,1,residential,26.799369,43.883001,1.0,1.0,0.0,1
1,9,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,1.0,POINT (-87.98060607910156 42.08369445800781),2,249.937500,-14.125000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083694,-87.980606,3813,0,1,secondary,35.678794,40.207796,0.0,0.0,0.0,1
2,11,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,POINT (-87.98035430908203 42.08379364013672),3,49.245537,-16.220982,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083794,-87.980354,3708,0,1,residential,19.033046,37.148259,1.0,2.0,0.0,1
3,14,Ho0laSMnEYrjIhh2Xk0xVg,SurfaceProblem,1.0,POINT (-87.98036193847656 42.08383560180664),2,352.198669,-14.926339,269.854431,0.359947,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083836,-87.980362,16186,0,1,secondary,29.462392,0.000000,1.0,1.0,0.0,1
4,18,VvFlgEkOuGmMtvvgO6lZog,CurbRamp,1.0,POINT (-87.97919464111328 42.08381652832031),2,40.319195,-12.854911,270.559692,1.520401,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083817,-87.979195,8414,1,3,residential,17.518682,37.016625,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352,21502,xKEHCYuha-gJIlbS69pQSQ,SurfaceProblem,2.0,POINT (-87.66431427001953 41.611446380615234),1,128.779022,-34.812500,179.218628,-0.064674,71959a82-3f47-4dec-a02b-86669bf8095b,41.611446,-87.664314,14581,0,1,residential,27.643323,0.000000,1.0,2.0,0.0,1
5353,21505,azH3XAm4QKnRlWXtJaiwxA,Obstacle,3.0,POINT (-87.66429901123047 41.61117935180664),1,156.529022,-31.625000,179.852203,0.201590,71959a82-3f47-4dec-a02b-86669bf8095b,41.611179,-87.664299,12602,0,1,residential,32.250867,0.000000,1.0,1.0,0.0,1
5354,21506,azH3XAm4QKnRlWXtJaiwxA,SurfaceProblem,2.0,POINT (-87.66453552246094 41.611183166503906),1,225.716522,-35.000000,179.852203,0.201590,71959a82-3f47-4dec-a02b-86669bf8095b,41.611183,-87.664536,14538,0,1,residential,32.434463,0.000000,1.0,1.0,0.0,1
5355,21507,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,3.0,POINT (-87.66429138183594 41.61117935180664),1,126.341515,-32.937500,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611179,-87.664291,14528,0,1,residential,34.337291,0.000000,1.0,1.0,0.0,1


In [19]:
#if the label_id in gt take it out of df
training = df[~df.label_id.isin(test.label_id)]

In [20]:
training

,label_id,gsv_panorama_id,label_type,severity,geometry,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
2,10,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,POINT (-87.98060607910156 42.08378601074219),3,288.656250,-13.187500,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083786,-87.980606,7014,0,1,secondary,36.434254,39.393508,1.0,1.0,0.0
4,12,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,POINT (-87.98036193847656 42.08381652832031),3,42.636162,-16.033482,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083817,-87.980362,7719,0,1,residential,27.337529,40.358872,1.0,2.0,0.0
5,13,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,POINT (-87.9805908203125 42.083824157714844),3,305.850433,-7.372768,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083824,-87.980591,6829,0,1,secondary,32.572773,43.586202,1.0,1.0,0.0
7,15,JNAmJAZYxrX7YSC4C3EwtA,SurfaceProblem,1.0,POINT (-87.97992706298828 42.08382034301758),3,33.604912,-15.194197,269.981445,0.073532,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083820,-87.979927,14517,0,1,residential,30.729777,0.000000,1.0,1.0,0.0
8,16,VvFlgEkOuGmMtvvgO6lZog,CurbRamp,1.0,POINT (-87.9793701171875 42.083770751953125),2,315.229919,-14.551339,270.559692,1.520401,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083771,-87.979370,6233,1,3,residential,15.248823,33.721013,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18276,21503,5XUiqq_KZD0TOFfl8RQ2RA,SurfaceProblem,2.0,POINT (-87.66429138183594 41.61152648925781),1,109.091515,-25.812500,179.677460,0.077150,71959a82-3f47-4dec-a02b-86669bf8095b,41.611526,-87.664291,14566,0,1,residential,33.772149,0.000000,1.0,1.0,0.0
18277,21504,oTenoeeETbFSoJ0k5C8bRA,SurfaceProblem,2.0,POINT (-87.66454315185547 41.61128616333008),1,213.716522,-33.687500,180.324783,0.074486,71959a82-3f47-4dec-a02b-86669bf8095b,41.611286,-87.664543,14554,0,1,residential,34.688455,0.000000,1.0,1.0,0.0
18281,21508,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,2.0,POINT (-87.66455078125 41.61113739013672),1,213.154022,-35.000000,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611137,-87.664551,14505,0,1,residential,36.532837,0.000000,1.0,1.0,0.0
18282,21509,tZWhmOw4CIP9bQLJycSyqQ,CurbRamp,1.0,POINT (-87.6463394165039 41.60987091064453),3,217.380630,-16.302500,179.386520,-0.660645,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,41.609871,-87.646339,516,1,3,residential,24.820652,45.526460,0.0,0.0,0.0


In [27]:
training.to_csv('../../../data/' + city + '/processed-labels/training_set_' + city + '.csv', index=False)
test.to_csv('../../../data/' + city + '/processed-labels/test_set_' + city + '.csv', index=False)
df.to_csv('../../../data/' + city + '/processed-labels/labels_full_features_' + city + '.csv', index=False)